In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/robi-datathon-30-qualifier-problem-2/problem 2.csv
/kaggle/input/robi-datathon-30-qualifier-problem-2/boxes.csv
/kaggle/input/robi-datathon-30-qualifier-problem-2/sample submission 2.csv
/kaggle/input/robi-datathon-30-qualifier-problem-2/purchase.csv


# Load Datasets

In [2]:
boxes_df = pd.read_csv('/kaggle/input/robi-datathon-30-qualifier-problem-2/boxes.csv')
problem_2_df = pd.read_csv('/kaggle/input/robi-datathon-30-qualifier-problem-2/problem 2.csv')
purchase_df = pd.read_csv('/kaggle/input/robi-datathon-30-qualifier-problem-2/purchase.csv')

sample_submission_2 = pd.read_csv('/kaggle/input/robi-datathon-30-qualifier-problem-2/sample submission 2.csv')

In [3]:
boxes_df

,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE
0,1,Premium,Home Delivery - CoD,0.0,2.7,9.96
1,2,Premium,Home Delivery - CoD,0.0,2.3,11.96
2,3,Premium,Home Delivery - CoD,0.0,2.4,11.96
3,4,Premium,Home Delivery - CoD,0.0,2.5,11.96
4,5,Premium,Home Delivery - CoD,0.0,2.6,11.96
...,...,...,...,...,...,...
285,286,Standard,Home Delivery - Digital Payment,0.0,4.0,15.96
286,287,Standard,Home Delivery - Digital Payment,0.0,2.9,15.96
287,288,Standard,Home Delivery - Digital Payment,0.0,3.6,15.96
288,289,Standard,Home Delivery - Digital Payment,0.0,4.7,15.96


In [4]:
boxes_df.isnull().sum().sum()

0

In [5]:
problem_2_df

,MAGIC_KEY
0,2C0B5F97180
1,2C2F659EF00
2,29D629EDF20
3,2A7073E2FF0
4,2A9B9BE1210
...,...
43513,2C49A3D7EAD
43514,2C49C75F0D8
43515,2C49CA2A632
43516,2C49D52753F


In [6]:
problem_2_df.isnull().sum().sum()

0

In [7]:
purchase_df

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT
0,1/2/2019,2CED678A247,12.0,1.0
1,1/2/2019,2BF58D91BA1,12.0,1.0
2,1/2/2019,2C15B86534E,99.0,1.0
3,1/2/2019,2C32D9A859A,6.0,1.0
4,1/2/2019,2C7A55404D1,4.0,1.0
...,...,...,...,...
2455859,28/10/2018,2BD992B5538,12.0,1.0
2455860,28/10/2018,2C97CD72233,17.0,1.0
2455861,28/10/2018,2C91C61D372,40.0,1.0
2455862,28/10/2018,2CD70CFC4E3,51.0,1.0


In [8]:
purchase_df.isnull().sum().sum()

94

In [9]:
sample_submission_2

,MAGIC_KEY,BOX_ID
0,2C0B5F97180,230
1,2C2F659EF00,230
2,29D629EDF20,230
3,2A7073E2FF0,230
4,2A9B9BE1210,230
...,...,...
43513,2C49A3D7EAD,230
43514,2C49C75F0D8,230
43515,2C49CA2A632,230
43516,2C49D52753F,230


In [10]:
sample_submission_2['BOX_ID'].unique()

array([230])

In [11]:
sample_submission_2.isnull().sum().sum()

0

In [12]:
(problem_2_df['MAGIC_KEY'] == sample_submission_2['MAGIC_KEY']).unique()

array([ True])

# 1. EDA

In [13]:
merged_df = pd.merge(boxes_df, purchase_df, on='BOX_ID', how='inner')
merged_df

,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,PURCHASE_DATE,MAGIC_KEY,BOX_COUNT
0,1,Premium,Home Delivery - CoD,0.0,2.7,9.96,4/2/2019,2C88D36D1FC,1.0
1,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2/1/2019,2BF011BDB38,1.0
2,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2/1/2019,2CA0EE8F2B3,1.0
3,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2/1/2019,2C623730B09,1.0
4,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2/1/2019,2CA6CE1054F,1.0
...,...,...,...,...,...,...,...,...,...
2455795,289,Standard,Home Delivery - Digital Payment,0.0,4.7,15.96,18/2/2019,29149CE828C,1.0
2455796,290,Standard,Home Delivery - Digital Payment,12.0,1.8,19.98,20/2/2019,28FF7857D43,1.0
2455797,290,Standard,Home Delivery - Digital Payment,12.0,1.8,19.98,5/1/2019,28F675D813E,1.0
2455798,290,Standard,Home Delivery - Digital Payment,12.0,1.8,19.98,11/1/2019,29D99F43873,1.0


In [14]:
merged_df.isnull().sum()

BOX_ID             0
QUALITY            0
DELIVERY_OPTION    0
MILK               0
MEAT               0
UNIT_PRICE         0
PURCHASE_DATE      0
MAGIC_KEY          0
BOX_COUNT          0
dtype: int64

In [15]:
merged_df.duplicated().sum()

77

In [16]:
df_unique = merged_df.drop_duplicates(keep='last')
df_unique

,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,PURCHASE_DATE,MAGIC_KEY,BOX_COUNT
0,1,Premium,Home Delivery - CoD,0.0,2.7,9.96,4/2/2019,2C88D36D1FC,1.0
1,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2/1/2019,2BF011BDB38,1.0
2,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2/1/2019,2CA0EE8F2B3,1.0
3,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2/1/2019,2C623730B09,1.0
4,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2/1/2019,2CA6CE1054F,1.0
...,...,...,...,...,...,...,...,...,...
2455795,289,Standard,Home Delivery - Digital Payment,0.0,4.7,15.96,18/2/2019,29149CE828C,1.0
2455796,290,Standard,Home Delivery - Digital Payment,12.0,1.8,19.98,20/2/2019,28FF7857D43,1.0
2455797,290,Standard,Home Delivery - Digital Payment,12.0,1.8,19.98,5/1/2019,28F675D813E,1.0
2455798,290,Standard,Home Delivery - Digital Payment,12.0,1.8,19.98,11/1/2019,29D99F43873,1.0


In [17]:
df_dropped = df_unique.drop(['PURCHASE_DATE', 'BOX_COUNT'], axis=1)
df_dropped

,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,MAGIC_KEY
0,1,Premium,Home Delivery - CoD,0.0,2.7,9.96,2C88D36D1FC
1,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2BF011BDB38
2,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2CA0EE8F2B3
3,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2C623730B09
4,2,Premium,Home Delivery - CoD,0.0,2.3,11.96,2CA6CE1054F
...,...,...,...,...,...,...,...
2455795,289,Standard,Home Delivery - Digital Payment,0.0,4.7,15.96,29149CE828C
2455796,290,Standard,Home Delivery - Digital Payment,12.0,1.8,19.98,28FF7857D43
2455797,290,Standard,Home Delivery - Digital Payment,12.0,1.8,19.98,28F675D813E
2455798,290,Standard,Home Delivery - Digital Payment,12.0,1.8,19.98,29D99F43873


In [18]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df_dropped['QUALITY'] = encoder.fit_transform(df_dropped['QUALITY'])
df_dropped['DELIVERY_OPTION'] = encoder.fit_transform(df_dropped['DELIVERY_OPTION'])
df_dropped

,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,MAGIC_KEY
0,1,0,1,0.0,2.7,9.96,2C88D36D1FC
1,2,0,1,0.0,2.3,11.96,2BF011BDB38
2,2,0,1,0.0,2.3,11.96,2CA0EE8F2B3
3,2,0,1,0.0,2.3,11.96,2C623730B09
4,2,0,1,0.0,2.3,11.96,2CA6CE1054F
...,...,...,...,...,...,...,...
2455795,289,1,2,0.0,4.7,15.96,29149CE828C
2455796,290,1,2,12.0,1.8,19.98,28FF7857D43
2455797,290,1,2,12.0,1.8,19.98,28F675D813E
2455798,290,1,2,12.0,1.8,19.98,29D99F43873


In [19]:
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2455723 entries, 0 to 2455799
Data columns (total 7 columns):
 #   Column           Dtype  
---  ------           -----  
 0   BOX_ID           int64  
 1   QUALITY          int64  
 2   DELIVERY_OPTION  int64  
 3   MILK             float64
 4   MEAT             float64
 5   UNIT_PRICE       float64
 6   MAGIC_KEY        object 
dtypes: float64(3), int64(3), object(1)
memory usage: 149.9+ MB


In [20]:
sub_test_df = df_dropped[df_dropped['MAGIC_KEY'].isin(problem_2_df['MAGIC_KEY'])]
sub_test_df

,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,MAGIC_KEY
1,2,0,1,0.0,2.3,11.96,2BF011BDB38
158,2,0,1,0.0,2.3,11.96,2BCFA1C20C2
427,2,0,1,0.0,2.3,11.96,2BE383C74BC
513,2,0,1,0.0,2.3,11.96,2C07292B30D
553,2,0,1,0.0,2.3,11.96,2C2D4CDC611
...,...,...,...,...,...,...,...
2455344,278,1,2,8.5,0.0,10.14,29D0293C978
2455441,278,1,2,8.5,0.0,10.14,29D0293C978
2455509,280,1,2,10.2,0.0,12.18,2A13A071D93
2455582,281,1,2,10.7,0.0,12.78,29E43468DA2


In [21]:
sub_test_df = sub_test_df.drop_duplicates(subset='MAGIC_KEY', keep='last')
sub_test_df

,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,MAGIC_KEY
2257,3,0,1,0.0,2.4,11.96,2C2D4CDC611
2467,3,0,1,0.0,2.4,11.96,2BE383C74BC
5715,3,0,1,0.0,2.4,11.96,2C377E86612
5926,3,0,1,0.0,2.4,11.96,2BC42A953F4
6281,3,0,1,0.0,2.4,11.96,2C28EA36D52
...,...,...,...,...,...,...,...
2455264,278,1,2,8.5,0.0,10.14,29DF57C5686
2455326,278,1,2,8.5,0.0,10.14,29DDFF395CB
2455509,280,1,2,10.2,0.0,12.18,2A13A071D93
2455582,281,1,2,10.7,0.0,12.78,29E43468DA2


In [22]:

sub_test_df = sub_test_df.reset_index(drop=True)
sample_submission_2 = sample_submission_2.reset_index(drop=True)

sample_submission_2['MAGIC_KEY'] = sub_test_df['MAGIC_KEY']


In [23]:
sample_submission_2

,MAGIC_KEY,BOX_ID
0,2C2D4CDC611,230
1,2BE383C74BC,230
2,2C377E86612,230
3,2BC42A953F4,230
4,2C28EA36D52,230
...,...,...
43513,29DF57C5686,230
43514,29DDFF395CB,230
43515,2A13A071D93,230
43516,29E43468DA2,230


In [24]:
df_dropped.drop(['MAGIC_KEY'], axis=1, inplace=True)
sub_test_df.drop(['MAGIC_KEY'], axis=1, inplace=True)


df_dropped

,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE
0,1,0,1,0.0,2.7,9.96
1,2,0,1,0.0,2.3,11.96
2,2,0,1,0.0,2.3,11.96
3,2,0,1,0.0,2.3,11.96
4,2,0,1,0.0,2.3,11.96
...,...,...,...,...,...,...
2455795,289,1,2,0.0,4.7,15.96
2455796,290,1,2,12.0,1.8,19.98
2455797,290,1,2,12.0,1.8,19.98
2455798,290,1,2,12.0,1.8,19.98


In [25]:
X = df_dropped.drop(['BOX_ID'], axis=1)
y = df_dropped['BOX_ID']

sub_test_df = sub_test_df.drop(['BOX_ID'], axis=1)

X

,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE
0,0,1,0.0,2.7,9.96
1,0,1,0.0,2.3,11.96
2,0,1,0.0,2.3,11.96
3,0,1,0.0,2.3,11.96
4,0,1,0.0,2.3,11.96
...,...,...,...,...,...
2455795,1,2,0.0,4.7,15.96
2455796,1,2,12.0,1.8,19.98
2455797,1,2,12.0,1.8,19.98
2455798,1,2,12.0,1.8,19.98


# Normalizetions

In [26]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

X = scaler.fit_transform(X)
sub_test_df = scaler.fit_transform(sub_test_df)
X

array([[ 0.        ,  1.        , -0.90909091,  0.71428571, -1.33222591],
       [ 0.        ,  1.        , -0.90909091,  0.14285714, -1.        ],
       [ 0.        ,  1.        , -0.90909091,  0.14285714, -1.        ],
       ...,
       [ 1.        ,  2.        ,  0.18181818, -0.57142857,  0.33222591],
       [ 1.        ,  2.        ,  0.18181818, -0.57142857,  0.33222591],
       [ 1.        ,  2.        ,  0.18181818, -0.57142857,  0.33222591]])

# Split dataset

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier



classifiers = {
    'Logistic Regression': LogisticRegression(max_iter=200),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Support Vector Machine': SVC(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(n_estimators=100),
    'Gradient Boosting': GradientBoostingClassifier()
}


In [29]:
from sklearn.metrics import accuracy_score


accuracy_scores = {}
models = {}
best_model = None
best_accuracy = float('-inf')

for name, clf in classifiers.items():
    model = clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    if best_accuracy < accuracy:
        best_model = model
    
    models[name] = model
    accuracy_scores[name] = accuracy
    print(f"{name} Accuracy: {accuracy * 100:.2f}%")
    
    break


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 97.04%


In [30]:
best_model

LogisticRegression(max_iter=200)

# Submission

In [31]:
predict = best_model.predict(sub_test_df)
predict

array([3, 3, 3, ..., 8, 8, 8])

In [32]:
sample_submission_2['BOX_ID'] = predict

In [33]:
sample_submission_2

,MAGIC_KEY,BOX_ID
0,2C2D4CDC611,3
1,2BE383C74BC,3
2,2C377E86612,3
3,2BC42A953F4,3
4,2C28EA36D52,3
...,...,...
43513,29DF57C5686,8
43514,29DDFF395CB,8
43515,2A13A071D93,8
43516,29E43468DA2,8


In [34]:
sample_submission_2.to_csv('submission.csv', index=False)